<br><center><span style='font-family: "DejaVu Sans"; font-size:48px; color:#FF7133'>Growing Pains Case</span></center>
<center><span style='font-family: "DejaVu Sans"; font-size:28px'><i>Processing Solution Results</i></span></center>

---

In [1]:
# Standard libraries
from __future__ import print_function
import os
import pickle
from copy import deepcopy
from itertools import product, permutations, combinations
from collections import defaultdict
# 3rd party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipyleaflet import Map, Marker, MarkerCluster, AwesomeIcon, AntPath

---

Import the file with outputs from the solver:

In [2]:
soln_file = 'pickle/GPCaseResults_24Aug20_18-42.pickle'
data_file = 'pickle/gp_data.pickle'

with open(soln_file, 'rb') as f:
    results = pickle.load(f)
    
with open(data_file, 'rb') as f:
    gp_data = pickle.load(f)

In [3]:
time_matrices = gp_data[0]  # daily travel time matrices 
demands = gp_data[1]  # demand per location per day
travel_times = gp_data[2]  # matrix of travel times between ALL locations (dataframe)
daily_orders = gp_data[3]  # dataframe of daily orders for reference
daily_total_dels = gp_data[4]  # summary of total orders by day
daily_total_vols = gp_data[5]  # summary of total volume required by day

<br> Import the store locations and process as was done before in the [EDA notebook](GrowingPains_EDA_DataPrep.ipynb).

In [5]:
# Store locations
store_locs = pd.read_csv('data/locations.csv', nrows=123)  # last few rows in .csv are junk
# Change zipcodes to strings and pad with leading zeros
store_locs['ZIP'] = store_locs['ZIP'].apply(lambda x: str(int(x)).zfill(5)) 
store_locs.head(10)

,ZIP,X,Y,CITY,STATE,ZIPID
0,01060,-72.631389,42.318611,Northampton,MA,1
1,01101,-72.578056,42.106111,Springfield,MA,2
2,01420,-71.802222,42.583611,Fitchburg,MA,3
3,01510,-71.682778,42.416667,Clinton,MA,4
4,01570,-71.885556,42.048056,Webster,MA,5
5,01581,-71.615833,42.269444,Westborough,MA,6
6,01606,-71.819167,42.292500,Worcester,MA,7
7,01701,-71.415833,42.276944,Framingham,MA,8
8,01730,-71.276667,42.491389,Bedford,MA,9
9,01752,-71.540556,42.346111,Marlborough,MA,10


The statement below from the EDA and pre-processing notebook converts the zip ID's to coordinates that can be provided as input to an interactive map.

In [6]:
store_loc_coords = store_locs.loc[:, ['ZIPID', 'Y', 'X']].set_index('ZIPID').T.to_dict('list')
# store_loc_coords

---

<br> <h3><u>Result Summaries</u></h3>

The function below summarizes the overall totals making it easy to see at a glance the requirements for each day and compare different models.

In [7]:
days = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri']
totals = ['totalNumberOfVehicles', 
          'totalLoadAllVehicles', 
          'totalTimeAllVehicles', 
          'totalDistanceAllVehicles']

def result_summary(results=results, days=days, totals=totals):
    results = deepcopy(results)
    summary_dict = dict()
    for day in days:
        summary_dict[day] = {total: results.get(day).get(total) for total in totals}
        
    return pd.DataFrame(summary_dict)

In [8]:
result_summary()

,Mon,Tue,Wed,Thu,Fri
totalNumberOfVehicles,4,5,6,6,6
totalLoadAllVehicles,10223,11249,15192,15009,13468
totalTimeAllVehicles,3240,3657,3183,3590,3378
totalDistanceAllVehicles,1459,1647,1267,1463,1394


Loads are in cubic <i>ft<sup>3</sup></i> volume, total time is in <i>minutes</i>, and total distance is in <i>miles</i>.

---

<br> <h3><u>Schedules</u></h3>

The ultimate goal of this problem is to obtain schedules for each day. The class below processes the data from `results` and returns a summary for all the day's routes and a detailed schedule for every route.

The first two functions are the functions from the [solution notebook](GrowingPains_ProblemSolution.ipynb) to convert the times from minutes to clock time:

In [9]:
def clock_to_minutes(clock_time):
    '''
    Converts clock time to time of day in minutes
    '''
    t = clock_time.split(':')
    return (int(t[0])*60 + int(t[1]))


def minutes_to_clock(time_in_minutes):
    '''
    Converts minutes of the day to clock time
    '''
    return ('{:02d}:{:02d}'.format(*divmod(time_in_minutes, 60)))

<br> This class returns the summaries for all routes in a day and a schedule for each route.

In [10]:
class route_scheduler(object):
    '''
    Returns route schedules from the solution results
    '''
    def __init__(self, day, results=results, demands=demands):
        '''
        day: day to schedule
        results: The dict with solution results
        demands: Dict with demands and orignal ZIPIDs
        '''
        self.day = day
        # Important! Use deepcopy() else things can get messy
        self.results = deepcopy(results.get(self.day))
        self.demands = deepcopy(demands)
        self.vehicles = [veh for veh in self.results.keys() \
                         if veh[-1].isnumeric()] 
        # map result ID's back to original ID's
        zipIDs = list(self.demands.get(self.day).keys())[:]
        # make sure depot (DC - ZIPID 20) is included and comes first
        if 20 not in zipIDs:
            zipIDs.insert(0, 20)
        if zipIDs[0] != 20:
            zipIDs.remove(20)
            zipIDs.insert(0, 20)
        self.stop_labels = dict(enumerate(zipIDs))
    
    def get_vehicles(self):
        '''Returns vehicle labels'''
        return self.vehicles
        
    
    def route_summaries(self):
        '''
        Get route summaries for given day
        '''
        summary = defaultdict(dict)
        for vh in self.vehicles:
            t = self.results.get(vh).get('totalTime')
            d = self.results.get(vh).get('totalDistance')
            L = self.results.get(vh).get('totalLoad')
            summary[vh] = {'Total Duty Time': f'{t//60}hrs {t%60}min', 
                           'Total Travel Distance': f'{d} miles', 
                           'Total Driving Time': f'{d//40}hrs {int(((d%40)/40)*60)}min', 
                           'Load Carried': f'{L} cuft.'}
        
        return pd.DataFrame(summary)
    
    
    def vehicle_schedule(self, vehicle):
        '''Get complete schedule for a route'''
        
        vh = self.results.get(vehicle)
        
        stop_idx = vh.get('stops')
        stops = []
        for idx in stop_idx:
            z = self.stop_labels.get(idx)  # get ZIPID
            cty = store_locs[store_locs.ZIPID == z].get('CITY').to_numpy()[0]
            st = store_locs[store_locs.ZIPID == z].get('STATE').to_numpy()[0]
            stop = "Distribution Center" if idx == 0 else " ".join([cty, st])
            stops.append(stop)
        # arrival times
        t_arr = []
        for i,t in enumerate(vh.get('arrive')):
            if i == 0:
                t_arr.append(None)  # depot
            else:
                t_arr.append(minutes_to_clock(t))
        # unloading time
        unload = [(j-i) for i,j in\
                  zip(vh.get('arrive')[:-1], vh.get('depart'))]
        unload.append(0)  # add zero for end point (depot)
        # deliveries
        deliveries = vh.get('load')[:]
        deliveries.append(0)  # add zero for end point (depot)
        # departures
        t_dep = []
        for t in vh.get('depart'):
            t_dep.append(minutes_to_clock(t))
        t_dep.append(None)  # depot
        # travel distance
        d_tot = vh.get('distance')
        dist = []
        for i in range(len(d_tot)):
            if i == 0:  # route start
                dist.append(d_tot[i])
            else:
                dist.append(d_tot[i] - d_tot[i-1])
                
        schedule = pd.DataFrame({'Stop': stops, 
                                 'Arrive': t_arr, 
                                 'Unloading_time(min)': unload, 
                                 'Departure': t_dep, 
                                 'Delivery(cuft.)': deliveries, 
                                 'Travel_distance(mi)': dist})
        
        return schedule

<br> <b>Using the Route Scheduler</b>
<br> An example showing route summaries and a few route schedules for <u>Thursday</u>:

<b>Summaries:</b>

In [11]:
Thu_schedule = route_scheduler('Thu')
Thu_schedule.route_summaries()

,vehicle_0,vehicle_2,vehicle_3,vehicle_4,vehicle_5,vehicle_7
Total Duty Time,9hrs 56min,2hrs 51min,13hrs 57min,10hrs 50min,8hrs 17min,13hrs 59min
Total Travel Distance,239 miles,42 miles,400 miles,267 miles,134 miles,381 miles
Total Driving Time,5hrs 58min,1hrs 3min,10hrs 0min,6hrs 40min,3hrs 21min,9hrs 31min
Load Carried,2643 cuft.,2820 cuft.,1773 cuft.,2924 cuft.,3007 cuft.,1842 cuft.


<b>Route schedules:</b>

In [12]:
# vehicle_2
Thu_schedule.vehicle_schedule('vehicle_2')

,Stop,Arrive,Unloading_time(min),Departure,Delivery(cuft.),Travel_distance(mi)
0,Distribution Center,None,0,07:51,0,0
1,Reading MA,08:00,30,08:30,206,6
2,Waltham MA,08:55,79,10:14,2614,17
3,Distribution Center,10:42,0,None,0,19


In [13]:
# vehicle_3
Thu_schedule.vehicle_schedule('vehicle_3')

,Stop,Arrive,Unloading_time(min),Departure,Delivery(cuft.),Travel_distance(mi)
0,Distribution Center,None,0,04:44,0,0
1,Meriden CT,08:00,30,08:30,225,131
2,Bethany CT,08:54,30,09:24,138,16
3,Norwalk CT,10:22,30,10:52,152,39
4,Norwalk CT,10:58,30,11:28,348,4
5,New Canaan CT,11:40,30,12:10,195,8
6,Stamford CT,12:23,30,12:53,173,9
7,Ridgefield CT,13:15,30,13:45,125,15
8,Danbury CT,13:55,30,14:25,417,7
9,Distribution Center,18:41,0,None,0,171


In [14]:
# vehicle_4
Thu_schedule.vehicle_schedule('vehicle_4')

,Stop,Arrive,Unloading_time(min),Departure,Delivery(cuft.),Travel_distance(mi)
0,Distribution Center,None,0,07:06,0,0
1,Manchester NH,08:00,30,08:30,257,36
2,Lebanon NH,10:21,30,10:51,193,74
3,Hanover NH,11:13,30,11:43,305,15
4,Manchester NH,13:44,30,14:14,94,81
5,Manchester NH,14:18,30,14:48,197,3
6,Merrimack NH,15:06,30,15:36,319,12
7,Westford MA,16:10,30,16:40,163,23
8,Billerica MA,17:02,42,17:44,1396,15
9,Distribution Center,17:56,0,None,0,8


---

<br> <h3><u>Visualizing Routes</u></h3>

Visualizing the routes can be very helpful. This next class is used to display a map showing the locations visited by a given vehicle for a given day. The path on the map shows the connections and order of the stops.

In [15]:
if not os.path.exists('RouteMaps'):
    os.makedirs('RouteMaps')

In [16]:
class route_map(object):
    '''
    Create a map object to show routes
    '''
    def __init__(self, day, 
                 locations=store_loc_coords, 
                 demands=demands,  
                 results=results):
        '''
        day: Day to get routes
        locations: dict with delivery location coordinates
        results: dict object with solver results
        '''
        self.day = deepcopy(day)
        self.locations = deepcopy(locations)
        self.demands = deepcopy(demands)
        self.results = deepcopy(results.get(self.day))
        self.vehicles = [veh for veh in self.results.keys() \
                         if veh[-1].isnumeric()] 
        # map result ID's back to original ID's
        zipIDs = list(self.demands.get(self.day).keys())[:]
        # make sure depot (DC - ZIPID 20) is included and comes first
        if 20 not in zipIDs:
            zipIDs.insert(0, 20)
        if zipIDs[0] != 20:
            zipIDs.remove(20)
            zipIDs.insert(0, 20)
        self.stop_labels = dict(enumerate(zipIDs))
        
    
    def __repr__(self):
        return f'Show optimized routes for {day!r} on interactive map'
        

    def getAllStopCoords(self):
        '''
        Maps node IDs for all stops of given day 
        to map coordinates of store
        '''
        self.stop_coords = {idx:tuple(self.locations.get(zipID)) \
                            for idx, zipID in self.stop_labels.items()}
                

    def get_route_coords(self, vehicle):
        '''Get the route XY coords'''
        route = self.results.get(vehicle).get('stops')
        return dict(enumerate([self.stop_coords.get(stop) for stop in route]))

    
    def plot_route(self, vehicle, save_to_file=None):
        '''
        Plot route on an interactive map
        ---
        save: path and filename to save html map
        '''
        route = self.get_route_coords(vehicle)
        dc = route.get(0)
        dc_icon = AwesomeIcon(name='home', marker_color='red')
        # create map; centered roughly at mean of locations
        _ = list(zip(*list(route.values())))
        x, y = np.mean(_[0]), np.mean(_[1])
        route_map = Map(center=(x, y), zoom=8)
        # mark DC location
        dc_marker = Marker(location=dc, 
                           title='Depot-DC', 
                           icon=dc_icon, 
                           draggable=False)
        route_map.add_layer(dc_marker)
        # mark stops (1st and last are DC)
        stops = list(route.values())[1:-1]
        for i, loc in enumerate(stops):
            marker = Marker(location=loc, title=str(i), draggable=False)
            route_map.add_layer(marker)
        # show route path
        p = list(route.values())
        p.append(dc)
        path = AntPath(locations=p, weight=4)
        route_map.add_layer(path)
        
        if save_to_file is not None:
            route_map.save(save_to_file, title=str(self.day) + '_' + str(vehicle))
        
        return route_map

<br> <b>Some route examples:</b>

In [17]:
Wed_routes = route_scheduler('Wed')
Wed_routes.route_summaries()

,vehicle_0,vehicle_1,vehicle_2,vehicle_3,vehicle_6,vehicle_7
Total Duty Time,9hrs 8min,13hrs 28min,8hrs 36min,13hrs 56min,1hrs 42min,6hrs 13min
Total Travel Distance,207 miles,321 miles,220 miles,399 miles,12 miles,108 miles
Total Driving Time,5hrs 10min,8hrs 1min,5hrs 30min,9hrs 58min,0hrs 18min,2hrs 42min
Load Carried,1999 cuft.,2109 cuft.,3172 cuft.,2015 cuft.,2800 cuft.,3097 cuft.


In [18]:
Wed_routes.vehicle_schedule('vehicle_1')

,Stop,Arrive,Unloading_time(min),Departure,Delivery(cuft.),Travel_distance(mi)
0,Distribution Center,None,0,05:11,0,0
1,Northampton MA,08:00,30,08:30,151,113
2,Springfield MA,08:58,30,09:28,140,19
3,Windsor Locks CT,09:50,30,10:20,210,15
4,Windsor CT,10:27,30,10:57,94,5
5,Hartford CT,11:12,30,11:42,236,10
6,Hartford CT,11:43,30,12:13,149,1
7,Farmington CT,12:28,30,12:58,280,10
8,East Hartford CT,13:19,30,13:49,246,14
9,Glastonbury CT,14:02,30,14:32,140,9


In [19]:
Wed_map = route_map('Wed')
Wed_map.getAllStopCoords()

Wed_map.plot_route('vehicle_1', 'RouteMaps/Wed_veh1.html')

Map(center=[42.011089743589736, -72.3344230769231], controls=(ZoomControl(options=['position', 'zoom_in_text',…